# IndicCLIP Model

> Defines the main IndicCLIP model architecture, combining the vision and text encoders.

In [ ]:
#| default_exp model.clip

In [1]:
#| export
from pathlib import Path
try:
    import indic_clip.core
    print("Reloaded indic_clip.core")
except ModuleNotFoundError:
    print("indic_clip.core not found initially.")
    # Attempt to set sys.path if running in Colab and project cloned
    import sys
    if 'google.colab' in sys.modules:
        project_parent = '/content' # Assuming cloned into /content/indic-clip
        if Path('/content/drive/MyDrive/Indic-Clip').exists():
             project_parent = '/content/drive/MyDrive/Indic-Clip'
        if project_parent not in sys.path:
             sys.path.insert(0, project_parent)
             print(f"Added {project_parent} to sys.path")
        try:
            import indic_clip.core
            print("Imported indic_clip.core after path adjustment.")
        except ModuleNotFoundError:
            print("ERROR: Still cannot find indic_clip.core. Ensure project structure is correct.")
            print("Expected: /content/Indic-Clip/indic_clip/core.py or similar in Drive")
            # raise # Stop execution if core components missing

indic_clip.core not found initially.


In [ ]:
#| hide
%cd /content/drive/MyDrive/Indic-Clip/

In [ ]:
#| hide
!pip install -qr requirements.txt

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import logging

from fastai.vision.all import *

try:
    # Import necessary components from our project
    from indic_clip.core import get_logger, setup_logging, DEFAULT_EMBED_DIM, PRETRAINED_TOKENIZER_NAME
    from indic_clip.model.vision import VisionEncoder
    from indic_clip.model.text import TextEncoder
    from indic_clip.data.tokenization import IndicBERTTokenizer # Needed if passing tokenizer
except ModuleNotFoundError:
    print('MODULE NOT FOUND')
    # Fallback if core not found (e.g. testing)
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)
    def get_logger(name): return logging.getLogger(name)
    def setup_logging(): pass
    DEFAULT_EMBED_DIM = 768
    PRETRAINED_TOKENIZER_NAME = "ai4bharat/indic-bert"
    # Dummy classes if modules aren't available
    class VisionEncoder(nn.Module):
        def __init__(self, model_name='dummy_vision', pretrained=True, output_dim=None):
            super().__init__()
            self.feature_dim = 768
            self.backbone = nn.Identity()
            self.projection = nn.Linear(768, output_dim) if output_dim else nn.Identity()
        def forward(self, x): return self.projection(torch.randn(x.shape[0], self.feature_dim))
        def set_gradient_checkpointing(self, enable): pass

    class TextEncoder(nn.Module):
        def __init__(self, model_name='dummy_text', pretrained=True, output_dim=None, tokenizer=None):
            super().__init__()
            self.feature_dim = 768
            self.backbone = nn.Identity()
            self.projection = nn.Linear(768, output_dim) if output_dim else nn.Identity()
        def forward(self, input_ids, attention_mask): return self.projection(torch.randn(input_ids.shape[0], self.feature_dim))
        def set_gradient_checkpointing(self, enable): pass

    class IndicBERTTokenizer:
         def __init__(self, *args, **kwargs): self.vocab_size = 30000
         def tokenize(self, texts):
            if isinstance(texts, str): texts=[texts]
            ids = torch.randint(0, self.vocab_size, (len(texts), 10))
            mask = torch.ones_like(ids)
            return {'input_ids': ids, 'attention_mask': mask}
         @classmethod
         def load_tokenizer(cls, *args, **kwargs):
             return cls()


setup_logging()
logger = get_logger(__name__)

## IndicCLIP Model Definition

In [ ]:
#| export
class IndicCLIP(Module):
    """The main IndicCLIP model, combining Vision and Text Encoders.

    This module integrates image and text processing pipelines and projects
    their features into a shared embedding space for contrastive learning.
    """
    def __init__(self,
                 embed_dim: int = DEFAULT_EMBED_DIM,
                 vision_model_name: str = 'vit_base_patch16_224',
                 vision_pretrained: bool = True,
                 text_model_name: str = PRETRAINED_TOKENIZER_NAME, # Use tokenizer's base model
                 text_pretrained: bool = True,
                 tokenizer: IndicBERTTokenizer = None):
        """
        Initializes the IndicCLIP model.

        Args:
            embed_dim (int): The dimension of the shared embedding space.
            vision_model_name (str): Name of the timm vision model.
            vision_pretrained (bool): Whether to load pretrained weights for the vision model.
            text_model_name (str): Name or path of the Hugging Face text model.
            text_pretrained (bool): Whether to load pretrained weights for the text model.
            tokenizer (IndicBERTTokenizer): The tokenizer instance, needed for text encoder setup (embedding resize).
        """
        if tokenizer is None:
             logger.warning("No tokenizer provided to IndicCLIP. Text encoder might not resize embeddings correctly.")
             # Attempt to load a default one - this might fail if path isn't set up
             # from indic_clip.core import TOKENIZER_PATH
             # tokenizer = IndicBERTTokenizer.load_tokenizer(TOKENIZER_PATH)


        self.vision_encoder = VisionEncoder(
            model_name=vision_model_name,
            pretrained=vision_pretrained,
            output_dim=None # Projection handled below
        )

        self.text_encoder = TextEncoder(
            model_name=text_model_name,
            pretrained=text_pretrained,
            output_dim=None, # Projection handled below
            tokenizer=tokenizer # Pass tokenizer for potential embedding resize
        )

        # --- Projection Heads ---
        # Project features from vision/text backbones to the shared embed_dim
        if self.vision_encoder.feature_dim is None or self.text_encoder.feature_dim is None:
             raise ValueError("Could not determine feature dimensions for vision or text encoders.")

        self.visual_projection = nn.Linear(self.vision_encoder.feature_dim, embed_dim, bias=False)
        self.text_projection = nn.Linear(self.text_encoder.feature_dim, embed_dim, bias=False)

        # Initialize projection layers (optional, but common)
        # Often initialized to match CLIP's initialization if transferring
        # Default PyTorch init is Kaiming Uniform for Linear layers

        # --- Logit Scale ---
        # Learnable parameter for scaling similarity scores
        # Initialized according to OpenAI CLIP paper (log(1/0.07))
        self.logit_scale = nn.Parameter(torch.ones([]) * np.log(1 / 0.07))

        logger.info(f"IndicCLIP initialized with vision='{vision_model_name}', text='{text_model_name}', embed_dim={embed_dim}")

    def encode_image(self, image: torch.Tensor) -> torch.Tensor:
        # Check input image tensor (optional, but good practice)
        if torch.isnan(image).any() or torch.isinf(image).any():
            logger.error("!!! NaN/Inf detected in input image tensor to encode_image !!!")
            # Handle error - maybe return zeros? This depends on how you want to proceed.
            # Returning zeros might hide the problem later. Raising an error might be better.
            raise ValueError("NaN/Inf in input image tensor")
    
        image_features_raw = self.vision_encoder(image)
    
        # --- Check RAW features ---
        if torch.isnan(image_features_raw).any() or torch.isinf(image_features_raw).any():
            logger.error("!!! NaN/Inf detected in RAW image features from vision_encoder !!!")
            raise ValueError("NaN/Inf in raw image features") # Halt execution
    
        projected_features = self.visual_projection(image_features_raw)
    
        # --- Check PROJECTED features ---
        if torch.isnan(projected_features).any() or torch.isinf(projected_features).any():
            logger.error("!!! NaN/Inf detected in PROJECTED image features !!!")
            raise ValueError("NaN/Inf in projected image features") # Halt execution
    
        # --- Check NORM before normalization ---
        norms = projected_features.norm(p=2, dim=-1, keepdim=True)
        min_norm = norms.min().item()
        if min_norm < 1e-6: # Check for near-zero norms
            logger.warning(f"!!! Very small norm detected in projected image features before normalization: {min_norm} !!!")
            # Optionally: Find which item in the batch has the small norm
            problem_indices = torch.where(norms.squeeze() < 1e-6)[0]
            logger.warning(f"Problematic image batch indices: {problem_indices.tolist()}")
    
        # Normalize features
        normalized_features = F.normalize(projected_features, p=2, dim=-1)
    
        # --- Check FINAL normalized features ---
        if torch.isnan(normalized_features).any() or torch.isinf(normalized_features).any():
            logger.error(f"!!! NaN/Inf detected in FINAL NORMALIZED image features (min_norm was {min_norm}) !!!")
            raise ValueError("NaN/Inf in final normalized image features") # Halt execution

        return normalized_features

    def encode_text(self, input_ids: torch.Tensor, attention_mask: torch.Tensor) -> torch.Tensor:
        # Check input IDs/mask (optional)
        # ...
    
        text_features_raw = self.text_encoder(input_ids=input_ids, attention_mask=attention_mask)
    
        # --- Check RAW features ---
        if torch.isnan(text_features_raw).any() or torch.isinf(text_features_raw).any():
            logger.error("!!! NaN/Inf detected in RAW text features from text_encoder !!!")
            # Log input_ids that caused the issue if possible
            problem_input_ids = input_ids[torch.where(torch.isnan(text_features_raw).any(dim=1))[0]]
            logger.error(f"Problematic input_ids (sample): {problem_input_ids[:2]}")
            raise ValueError("NaN/Inf in raw text features")
    
        projected_features = self.text_projection(text_features_raw)
    
        # --- Check PROJECTED features ---
        if torch.isnan(projected_features).any() or torch.isinf(projected_features).any():
            logger.error("!!! NaN/Inf detected in PROJECTED text features !!!")
            raise ValueError("NaN/Inf in projected text features")
    
        # --- Check NORM before normalization ---
        norms = projected_features.norm(p=2, dim=-1, keepdim=True)
        min_norm = norms.min().item()
        if min_norm < 1e-6:
            logger.warning(f"!!! Very small norm detected in projected text features before normalization: {min_norm} !!!")
            problem_indices = torch.where(norms.squeeze() < 1e-6)[0]
            logger.warning(f"Problematic text batch indices: {problem_indices.tolist()}")
            # Optionally log the text corresponding to these indices if you have access to the original text batch
            # logger.warning(f"Problematic input_ids for low norm: {input_ids[problem_indices]}")
    
    
        # Normalize features
        normalized_features = F.normalize(projected_features, p=2, dim=-1)
    
        # --- Check FINAL normalized features ---
        if torch.isnan(normalized_features).any() or torch.isinf(normalized_features).any():
            logger.error(f"!!! NaN/Inf detected in FINAL NORMALIZED text features (min_norm was {min_norm}) !!!")
            raise ValueError("NaN/Inf in final normalized text features")
    
        return normalized_features

    def forward(self, image: torch.Tensor,
            text_input: tuple) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        # ... (unpack text_input) ...

        if isinstance(text_input, tuple) and len(text_input) == 2:
            logger.debug(f"Input text tuple shapes: ids={text_input[0].shape}, mask={text_input[1].shape}")
            logger.debug(f"Input text tuple dtypes: ids={text_input[0].dtype}, mask={text_input[1].dtype}")
            logger.debug(f"Input text tuple device: ids={text_input[0].device}, mask={text_input[1].device}")
        else:
            logger.error(f"MODEL FORWARD RECEIVED UNEXPECTED text_input type: {type(text_input)}")
            # Handle error - maybe raise? Returning dummy tensors can hide problems.
            raise TypeError("IndicCLIP forward expected a tuple for text_input")

        
        input_ids, attention_mask = text_input
    
        # --- Check logit_scale BEFORE encoding (should be stable here) ---
        current_logit_scale_val = self.logit_scale.data.item()
        if np.isnan(current_logit_scale_val) or np.isinf(current_logit_scale_val):
            logger.error(f"!!! Logit scale is NaN/Inf BEFORE encoding: {current_logit_scale_val} !!!")
            raise ValueError("Logit scale NaN/Inf before encoding")
    
        image_features = self.encode_image(image) # Will raise error if NaN occurs inside
        text_features = self.encode_text(input_ids, attention_mask) # Will raise error if NaN occurs inside
    
        # Clamp the logit scale parameter before exponentiating
        # Clamp log(1/T) to avoid T becoming too small (e.g., T > 0.01 -> log(1/T) < log(100))
        # Use a slightly stricter clamp? Maybe max=4.0 (exp(4) ~ 55)
        self.logit_scale.data.clamp_(min=-np.inf, max=np.log(1 / 0.01)) # max=4.605
        # Optional: Add a min clamp if worried about excessively large T? e.g., min=np.log(1/100) = -4.605
    
        logit_scale_val_after_clamp = self.logit_scale.data.item()
        if np.isnan(logit_scale_val_after_clamp) or np.isinf(logit_scale_val_after_clamp):
             logger.error(f"!!! Logit scale is NaN/Inf AFTER CLAMP: {logit_scale_val_after_clamp} !!!")
             raise ValueError("Logit scale NaN/Inf after clamp")
    
        # Return the exponentiated clamped value
        logit_scale_exp = self.logit_scale.exp()
    
        # --- Check final outputs ---
        if torch.isnan(image_features).any(): logger.error("NaN in image_features returned by forward")
        if torch.isnan(text_features).any(): logger.error("NaN in text_features returned by forward")
        if torch.isnan(logit_scale_exp): logger.error("NaN in logit_scale_exp returned by forward")


        if torch.isnan(image_features).any() or torch.isinf(image_features).any():
            logger.error("!!! FINAL CHECK: NaN/Inf in image_features before return !!!")
        
        # Potentially raise here again or return NaNs if you want the loss func to see them
        # raise ValueError("NaN/Inf in image_features before return")
        if torch.isnan(text_features).any() or torch.isinf(text_features).any():
            logger.error("!!! FINAL CHECK: NaN/Inf in text_features before return !!!")
        
        # raise ValueError("NaN/Inf in text_features before return")
        if torch.isnan(logit_scale_exp).any() or torch.isinf(logit_scale_exp):
            logger.error(f"!!! FINAL CHECK: NaN/Inf in logit_scale_exp before return: {logit_scale_exp.item()} !!!")
        
        # raise ValueError("NaN/Inf in logit_scale_exp before return")
        
        return image_features, text_features, logit_scale_exp

    def set_gradient_checkpointing(self, enable: bool = True):
        """Enable or disable gradient checkpointing for both encoders."""
        self.vision_encoder.set_gradient_checkpointing(enable)
        self.text_encoder.set_gradient_checkpointing(enable)
        logger.info(f"IndicCLIP gradient checkpointing {'enabled' if enable else 'disabled'} for both encoders.")

## Example Usage

In [ ]:
#| eval: false
if __name__ == '__main__':
    print("--- IndicCLIP Example --- ")
    try:
        # Ensure tokenizer is available (needed by TextEncoder init)
        # Use a simple instance for testing if not running full pipeline
        test_tokenizer = IndicBERTTokenizer.load_tokenizer()

        # 1. Instantiate the IndicCLIP model
        # Use smaller embed_dim for faster testing if needed
        model = IndicCLIP(
            embed_dim=768,
            vision_model_name='vit_base_patch16_224', # Or 'resnet18' for faster test
            vision_pretrained=True,
            text_model_name=PRETRAINED_TOKENIZER_NAME,
            text_pretrained=True,
            tokenizer=test_tokenizer # Pass the loaded tokenizer
        )
        model.eval() # Set to evaluation mode for testing
        print("Model Instantiated.")

        # 2. Create dummy inputs
        batch_size = 4
        img_size = 224 # Must match vision model expectation
        seq_len = 32

        dummy_images = torch.randn(batch_size, 3, img_size, img_size)
        # Use the actual tokenizer to create realistic token IDs
        dummy_texts = ["यह एक उदाहरण वाक्य है।"] * batch_size
        tokenized = test_tokenizer.tokenize(dummy_texts)
        dummy_input_ids = tokenized['input_ids'][:, :seq_len] # Truncate/pad if needed
        dummy_attn_mask = tokenized['attention_mask'][:, :seq_len]
        # Adjust shapes if tokenizer max_len is different
        if dummy_input_ids.shape[1] < seq_len:
            pad_len = seq_len - dummy_input_ids.shape[1]
            pad_tensor_ids = torch.full((batch_size, pad_len), test_tokenizer.pad_token_id or 0, dtype=torch.long)
            pad_tensor_mask = torch.zeros((batch_size, pad_len), dtype=torch.long)
            dummy_input_ids = torch.cat([dummy_input_ids, pad_tensor_ids], dim=1)
            dummy_attn_mask = torch.cat([dummy_attn_mask, pad_tensor_mask], dim=1)


        print(f"\nDummy Image Input Shape: {dummy_images.shape}")
        print(f"Dummy Text Input IDs Shape: {dummy_input_ids.shape}")
        print(f"Dummy Text Mask Shape: {dummy_attn_mask.shape}")

        # 3. Test forward pass
        print("\nTesting forward pass...")
        with torch.no_grad():
            img_feat, txt_feat, logit_val = model(dummy_images, dummy_input_ids, dummy_attn_mask)

        print(f"Forward pass output type: {type((img_feat, txt_feat, logit_val))}")
        print(f"Image Features Shape: {img_feat.shape}")
        print(f"Text Features Shape: {txt_feat.shape}")
        print(f"Logit Scale Value: {logit_val.item():.4f}...")

        assert img_feat.shape == (batch_size, model.visual_projection.out_features)
        assert txt_feat.shape == (batch_size, model.text_projection.out_features)
        assert img_feat.shape == txt_feat.shape
        assert logit_val.ndim == 0 # Scalar tensor

        # 4. Test individual encoders
        print("\nTesting encode_image...")
        with torch.no_grad():
            encoded_img = model.encode_image(dummy_images)
        print(f"Encoded Image Features Shape: {encoded_img.shape}")
        print(f"Encoded Image Features Norm (sum of squares): {encoded_img.norm(dim=-1).pow(2)}")
        assert encoded_img.shape == img_feat.shape
        # Check normalization (norm should be close to 1)
        assert torch.allclose(encoded_img.norm(dim=-1), torch.ones(batch_size), atol=1e-6)

        print("\nTesting encode_text...")
        with torch.no_grad():
            encoded_txt = model.encode_text(dummy_input_ids, dummy_attn_mask)
        print(f"Encoded Text Features Shape: {encoded_txt.shape}")
        print(f"Encoded Text Features Norm (sum of squares): {encoded_txt.norm(dim=-1).pow(2)}")
        assert encoded_txt.shape == txt_feat.shape
        # Check normalization
        assert torch.allclose(encoded_txt.norm(dim=-1), torch.ones(batch_size), atol=1e-6)

        print("\nIndicCLIP test completed successfully.")

    except Exception as e:
        print(f"An error occurred during IndicCLIP example: {e}")
        import traceback
        traceback.print_exc()

In [ ]:
#| hide
import nbdev
nbdev.nbdev_export() # Run this in terminal to export